In [1]:
import sys
sys.path.append('../bggcomplex')

from bggcomplex import BGGComplex
from fast_module import FastLieAlgebraCompositeModule, FastModuleFactory, BGGCohomology, WeightSet
from IPython.display import display, Math, Latex
import cohomology
import numpy as np

## Quantum center

In [2]:
def Mijk(BGG,i,j,k,subset=[]):
    factory = FastModuleFactory(BGG.LA)

    component_dic = {'u':factory.build_component('u','coad',subset=subset),
                    'g':factory.build_component('g','ad',subset=subset),
                     'n':factory.build_component('n','ad',subset=subset)}
    
    dim_n = len(factory.parabolic_n_basis(subset))
    
    components = []
    for r in range(j+k//2+1):
        if (j-r<=dim_n):
            components.append([('u',j+k//2-r,'sym'),('g',r,'wedge'),('n',j-r,'wedge')])

    module = FastLieAlgebraCompositeModule(factory,components,component_dic)
    
    return module

In [3]:
def sort_sign(A):
    num_cols = A.shape[1]
    if num_cols>1:
        signs=(-1)**(sum([(A[:,0]<A[:,i]) for i in range(1,num_cols)]))
        mask=sum([(A[:,0]==A[:,i]) for i in range(1,num_cols)])==0
        return (np.sort(A),signs,mask)
    else:
        return (A,np.ones(len(A),dtype=int),np.ones(len(A),dtype=bool))

In [4]:
def compute_phi(BGG,subset=[]):
    factory = FastModuleFactory(BGG.LA)
    
    component_dic = {'u':factory.build_component('u','coad',subset=subset)}
    components_phi = [[('u',1,'wedge')]]
    
    module_phi = FastLieAlgebraCompositeModule(factory,components_phi,component_dic)
    
    b_basis = factory.parabolic_p_basis(subset)
    u_basis = factory.parabolic_u_basis(subset)
    coad_dic = factory.coadjoint_action_tensor(b_basis,u_basis)
    
    phi_image = {b:[] for b in b_basis}
    for (b,u),n_dic in coad_dic.items():
        n,coeff = n_dic.items()[0]
        phi_image[b].append(np.array([factory.dual_root_dict[u],n,coeff]))
    
    for b in phi_image.keys():
        img = phi_image[b]
        if len(img)>0:
            phi_image[b]=np.vstack(img)
        else:
            phi_image[b]=[]
    return phi_image

In [5]:
def Tijk_basis(BGG,i,j,k,subset=[]):
    factory = FastModuleFactory(BGG.LA)
    wc_mod = Mijk(BGG,i,j,k,subset=subset)
    wc_rel = Mijk(BGG,i,j-1,k,subset=subset)
    
    coker_dic = dict()
    mu_source_counters = dict()
    
    phi_image = compute_phi(BGG,subset=subset)
    #phi_image = {i:[] for i,_ in phi_image.items()}
    
    for mu, components in wc_rel.weight_components.items():

        for b, n_tensor_u in phi_image.items():
            new_mu = tuple(mu+factory.weight_dic[b])
            if new_mu not in mu_source_counters:
                mu_source_counters[new_mu]=0
            sparse_relations = []
            for comp_num, basis in components:
                r = wc_rel.components[comp_num][1][1]
                t_un_insert = None
                t_g_insert = None
                for c, comp_list in enumerate(wc_mod.components):
                    if comp_list[1][1] == r:
                        t_un_insert = c
                    if comp_list[1][1] == r+1:
                        t_g_insert = c

                num_rows = len(basis)
                num_u,num_g,num_n = (j-1+k//2-r,r,j-1-r)

                basis_enum = np.arange(mu_source_counters[new_mu],mu_source_counters[new_mu]+num_rows,dtype=int).reshape((-1,1))
                mu_source_counters[new_mu]+=num_rows


                # Insert g
                if t_g_insert is not None:
                    new_basis = np.hstack([basis_enum,basis[:,:num_u],b*np.ones(shape=(num_rows,1),dtype=int),basis[:,num_u:]])
                    sort_basis, signs, mask = sort_sign(new_basis[:,num_u+1:num_u+num_g+2])
                    new_basis[:,num_u+1:num_u+num_g+2] = sort_basis
                    new_basis = new_basis[mask]
                    basis_g = np.zeros(shape=(len(new_basis),3), dtype=int)
                    for row_num, (row, sign) in enumerate(zip(new_basis, signs[mask])):
                        source = row[0]
                        target = wc_mod.weight_comp_index_numbers[new_mu][tuple(list(row[1:])+[t_g_insert])]
                        basis_g[row_num] = [source,target,-sign]
                    if len(basis_g)>0:
                        sparse_relations.append(basis_g)

                # Insert n_tensor_u
                if t_un_insert is not None:
                    for n,u,coeff in n_tensor_u:
                        n_column = n*np.ones(shape=(num_rows,1),dtype=int)
                        u_column = u*np.ones(shape=(num_rows,1),dtype=int)
                        new_basis = np.hstack([basis_enum,u_column,basis[:,:num_u+num_g],n_column,basis[:,num_u+num_g:]])
                        new_basis[:,1:num_u+2]=np.sort(new_basis[:,1:num_u+2])
                        sort_basis, signs, mask = sort_sign(new_basis[:,num_u+num_g+2:])
                        new_basis[:,num_u+num_g+2:]=sort_basis
                        signs=coeff*signs[mask]
                        new_basis = new_basis[mask]
                        basis_un = np.zeros(shape=(len(new_basis),3), dtype=int)
                        for row_num, (row, sign) in enumerate(zip(new_basis, signs)):
                            source = row[0]
                            target = wc_mod.weight_comp_index_numbers[new_mu][tuple(list(row[1:])+[t_un_insert])]
                            basis_un[row_num] = [source,target,sign]
                        if len(basis_un)>0:
                            sparse_relations.append(basis_un)
            if len(sparse_relations)>0:
                if new_mu not in coker_dic:
                    coker_dic[new_mu]=[]
                coker_dic[new_mu]+=sparse_relations
    coker_dic = {mu:cohomology.sort_merge(np.concatenate(rels)) for mu,rels in coker_dic.items()}

    T=dict()
    for mu,rels in coker_dic.items():
        source_dim = mu_source_counters[mu]
        target_dim = wc_mod.dimensions[mu]
        sparse_dic = dict()
        for source,target,coeff in rels:
            sparse_dic[(source,target)]=coeff

        M = matrix(ZZ,sparse_dic,nrows = source_dim, ncols = target_dim, sparse=True)
        T[mu]=M.right_kernel().basis_matrix()
    return T

In [6]:
def all_abijk(BGG,s=0,subset=[],half_only=True):
    dim_n = len(FastModuleFactory(BGG.LA).parabolic_n_basis(subset))
    output = []
    s_mod = s%2
    a_max = dim_n+(1+s_mod)%2
    for a_iterator in range(a_max):
        a = 2*a_iterator+s_mod
        for b in range(max(0,s-2*a),a+1):
            if (a+b) % 2 == 0:
                output.append((a,b,(a-b)//2,(a+b)//2,s-a))
    if half_only:
        new_out = []
        max_a = max(s[0] for s in output)
        for a,b,i,j,k in output:
            if a+b<=max_a:
                new_out.append((a,b,i,j,k))
        return sorted(new_out,key=lambda s: (s[0]+s[1],s[0],s[1]))
    else:
        return output

def display_ab_dic(ab_dic):
    a_set = set()
    b_set = set()
    for a,b in ab_dic.keys():
        a_set.add(a)
        b_set.add(b)
    a_set = sorted(a_set)
    b_set = sorted(b_set)
    column_string = r'{r|' + ' '.join('l'*len(a_set))+r'}'
    rows = list()
    for a in a_set:
        row_strings = [r'{\scriptstyle i+j='+str(a)+r'}']
        for b in b_set:
            if (a,b) in ab_dic:
                row_strings.append(ab_dic[(a,b)])
            else:
                row_strings.append('')
        rows.append(r'&'.join(row_strings))
    rows.append(r'\hline h^{i,j}&'+ '&'.join([r'{\scriptstyle j-i='+str(b)+r'}' for b in b_set]))
    all_strings = r'\\'.join(rows)
    display(Math(r'\begin{array}'+column_string+all_strings+r'\end{array}'))

In [7]:
from tqdm.notebook import tqdm
BGG = BGGComplex('A3')
ab_dic = dict()

subset=[1,2]

with tqdm(all_abijk(BGG,s=0,subset=subset)) as pbar:
    for a,b,i,j,k in pbar:
        pbar.set_description('i+j= %d, j-i = %d'%(a,b))
        cohom = BGGCohomology(BGG, Mijk(BGG,i,j,k,subset=subset), coker=Tijk_basis(BGG, i,j,k,subset=subset))
        cohom_string = cohom.cohomology_LaTeX(i=i,only_strings=True)
        ab_dic[(a,b)]=cohom_string

display_ab_dic(ab_dic)

((-1, 0, 0), 1)
[[11  0  0  1]
 [11  0  1 -1]]


[[ 0  0  1]
 [ 0  1 -1]]
((0, -1, 0), 1)
[[14  2  1 -1]
 [14  2  2  1]]


[[ 3  1 -1]
 [ 3  2  1]]
((0, 0, -1), 1)
[[10  0  0  1]
 [13  2  1 -1]
 [ 6  5  2  1]]


[[ 1  0  1]
 [ 2  1  1]
 [ 1  2 -1]
 [ 2  2 -1]]


total diff
[array([[ 0,  0,  1],
       [ 0,  1, -1],
       [ 3,  1, -1],
       [ 3,  2,  1],
       [ 1,  0,  1],
       [ 2,  1,  1],
       [ 1,  2, -1],
       [ 2,  2, -1]])]


total diff
[]
((-1, 0, -1), 1)
[[11  0  5  0 -1]
 [11  0  5  1  1]]


[[ 1  0 -1]
 [ 1  1  1]]
((0, -2, -1), 1)
[]


((0, -1, -2), 1)
[[10  0  2  0 -1]
 [12  0  5  0  1]
 [ 6  2  5  1  2]
 [ 7  2  5  1  1]]


[[ 0  0 -2]
 [ 0  1  2]]


total diff
[array([[ 1,  0, -1],
       [ 1,  1,  1],
       [ 0,  0, -2],
       [ 0,  1,  2]])]


total diff
[]



<IPython.core.display.Math object>

In [8]:
BGG = BGGComplex('B2')
s = 0
i = 0
j = 4
k = s-i-j
print('a=%d, b=%d, k=%d\n'%(i+j,j-i,k))

subset = []
cohom = BGGCohomology(BGG, Mijk(BGG,i,j,k,subset=subset), coker=Tijk_basis(BGG, i,j,k,subset=subset))
diff, dimspace = cohomology.compute_diff(cohom,(0,0),0)

a=4, b=4, k=-4

((-1, 0), 1)
[[6 7 0 1 2 1 1]
 [6 9 0 2 3 1 1]
 [6 5 1 2 3 2 1]
 [7 6 0 1 2 3 1]
 [9 6 0 2 3 3 1]
 [9 7 1 2 3 4 1]
 [7 9 1 2 3 4 1]]


[[1 1 1]
 [2 1 1]
 [3 3 1]
 [5 3 1]
 [4 4 1]
 [6 4 1]]
((-1, 0), 2)
[[ 3  6  1  2  6  1]
 [ 1  6  2  3  6 -1]
 [ 2  6  1  3  7  2]
 [ 4  7  1  2  8  1]
 [ 2  7  2  3  8  2]
 [ 4  9  2  3  8  1]
 [ 2  6  1  3  9 -2]
 [ 5  7  1  2 10  1]
 [ 2  7  2  3 10 -2]
 [ 5  9  2  3 10  1]
 [ 6  9  0  1 11  1]
 [ 5  6  0  2 12 -1]
 [ 6  9  0  1 13 -1]
 [ 5  7  1  2 14 -1]
 [ 8  9  1  2 14 -1]
 [ 8  9  1  2 16  1]
 [ 5  9  2  3 16  1]]


[[ 1  1  4]
 [ 2  1  4]
 [ 3  3  4]
 [ 5  3  4]
 [ 4  4  1]
 [ 6  4  1]
 [ 3  5 -2]
 [ 5  5 -2]
 [ 7  6  1]
 [ 3  7 -1]
 [ 5  7 -1]
 [ 7  8 -1]
 [ 6  9 -1]
 [ 9  9 -1]
 [ 4 11 -1]
 [ 9 11  1]]
((0, -1), 1)
[[ 7  6  0  1  3  0  2]
 [ 6  7  0  1  3  0  1]
 [ 6  7  0  1  3  1  1]
 [ 7  7  0  2  3  1  2]
 [ 6  8  0  2  3  1 -2]
 [ 6  8  0  2  3  2 -2]
 [ 7  8  1  2  3  2  2]
 [ 7  6  0  1  3  3  1]
 [ 8  6  0  2  3  3 -1]

In [9]:
diff.transpose()

[  0   0   0   0   0   0   0   0   4   0   8   0   0  -8   4   0   0   0   0]
[  4   4   0   0   0   0   0   0   4  -8   0   8   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0  -2   0   0   2   0   0   0   0   0   0]
[  0   0   4   0   4   0   0   0   0   0   4   4   0  -1  -2   0   0   0   0]
[  0   0   0   1   0   1   0   0   0   0   0  -2  -2   0  -1   0   0   0   0]
[  0   0  -2   0  -2   0   0   0   4   0   6  -2   0 -16   9   0   0   0   0]
[  0   0   0   0   0   0   1   0   0   0   0   0   0   0   0  -2   0   0   0]
[  0   0  -1   0  -1   0   0   0   0   0   0   0   0   0   0   1   1   0   0]
[  0   0   0   0   0   0  -1   0   0   0   0   0   0   2   0   0   0   1   0]
[  0   0   0   0   0  -1   0  -1   0   0   0   0   0   0   0   0  -2   0   0]
[  0   0   0   0   0   0   0   0   2   0   0   0   0   0   2   0   0  -2  -2]
[  0   0   0  -1   0   0   0   1   0   2   0   0   0   0   0   0   0   0   1]

In [10]:
diff.nrows()

19

In [12]:
module = Mijk(BGG,0,4,-4)
module.weight_components[(0,0)]

[(1, array([[6, 6, 0, 1, 3],
         [6, 7, 0, 2, 3],
         [6, 8, 1, 2, 3],
         [7, 6, 0, 2, 3],
         [7, 7, 1, 2, 3],
         [8, 6, 1, 2, 3]], dtype=int32)), (2, array([[3, 6, 2, 3],
         [4, 6, 1, 3],
         [4, 7, 2, 3],
         [5, 6, 1, 3],
         [5, 7, 2, 3],
         [6, 7, 0, 1],
         [6, 8, 0, 2],
         [6, 9, 0, 3],
         [7, 8, 1, 2],
         [7, 9, 1, 3],
         [8, 9, 2, 3]], dtype=int32))]

In [13]:
module.dimensions_components[0][(0,0)]
module.dimensions_components[1][(0,0)]

6

In [14]:
module.weight_comp_index_numbers[(0,0)]

{(3, 6, 2, 3, 2): 6,
 (4, 6, 1, 3, 2): 7,
 (4, 7, 2, 3, 2): 8,
 (5, 6, 1, 3, 2): 9,
 (5, 7, 2, 3, 2): 10,
 (6, 6, 0, 1, 3, 1): 0,
 (6, 7, 0, 1, 2): 11,
 (6, 7, 0, 2, 3, 1): 1,
 (6, 8, 0, 2, 2): 12,
 (6, 8, 1, 2, 3, 1): 2,
 (6, 9, 0, 3, 2): 13,
 (7, 6, 0, 2, 3, 1): 3,
 (7, 7, 1, 2, 3, 1): 4,
 (7, 8, 1, 2, 2): 14,
 (7, 9, 1, 3, 2): 15,
 (8, 6, 1, 2, 3, 1): 5,
 (8, 9, 2, 3, 2): 16}

In [15]:
BGG = BGGComplex('B2')
factory = FastModuleFactory(BGG.LA)
factory.weight_dic

{0: array([-1, -2], dtype=int32),
 1: array([-1, -1], dtype=int32),
 2: array([-1,  0], dtype=int32),
 3: array([ 0, -1], dtype=int32),
 4: array([0, 0], dtype=int32),
 5: array([0, 0], dtype=int32),
 6: array([1, 2], dtype=int32),
 7: array([1, 1], dtype=int32),
 8: array([1, 0], dtype=int32),
 9: array([0, 1], dtype=int32)}

In [16]:
compute_phi(BGG)

{0: [],
 1: array([[0, 9, 2]]),
 2: array([[ 1,  9, -1]]),
 3: array([[ 0,  7, -2],
        [ 1,  8,  1]]),
 4: array([[ 3,  9,  2],
        [ 0,  6,  2],
        [ 2,  8, -2]]),
 5: array([[ 3,  9, -1],
        [ 1,  7,  1],
        [ 2,  8,  2]])}

In [17]:
Mijk(BGG,0,3,-4).weight_components

{(-3, -5): [(1, array([[0, 0, 1]], dtype=int32))],
 (-3, -4): [(1, array([[0, 0, 2],
          [1, 0, 1]], dtype=int32))],
 (-3, -3): [(1, array([[0, 1, 2],
          [1, 0, 2],
          [2, 0, 1]], dtype=int32))],
 (-3, -2): [(0, array([[9, 0, 1, 2]], dtype=int32)), (1, array([[1, 1, 2],
          [2, 0, 2]], dtype=int32))],
 (-3, -1): [(1, array([[2, 1, 2]], dtype=int32))],
 (-2, -5): [(1, array([[0, 0, 3]], dtype=int32))],
 (-2, -4): [(1, array([[0, 1, 3],
          [1, 0, 3],
          [3, 0, 1]], dtype=int32))],
 (-2, -3): [(0, array([[8, 0, 1, 2],
          [9, 0, 1, 3]], dtype=int32)), (1, array([[0, 2, 3],
          [1, 1, 3],
          [2, 0, 3],
          [3, 0, 2],
          [4, 0, 1],
          [5, 0, 1]], dtype=int32))],
 (-2, -2): [(0, array([[7, 0, 1, 2],
          [9, 0, 2, 3]], dtype=int32)), (1, array([[1, 2, 3],
          [2, 1, 3],
          [3, 1, 2],
          [4, 0, 2],
          [5, 0, 2],
          [9, 0, 1]], dtype=int32))],
 (-2, -1): [(0, array([[6, 0, 1, 2

In [18]:
module.weight_comp_index_numbers[(-1,0)]

{(1, 6, 2, 3, 2): 9,
 (2, 6, 1, 3, 2): 10,
 (2, 7, 2, 3, 2): 11,
 (3, 6, 1, 2, 2): 12,
 (4, 6, 0, 2, 2): 13,
 (4, 7, 1, 2, 2): 14,
 (4, 9, 2, 3, 2): 15,
 (5, 6, 0, 2, 2): 16,
 (5, 7, 1, 2, 2): 17,
 (5, 9, 2, 3, 2): 18,
 (6, 4, 1, 2, 3, 1): 1,
 (6, 5, 1, 2, 3, 1): 2,
 (6, 6, 0, 1, 2, 3, 0): 0,
 (6, 7, 0, 1, 2, 1): 3,
 (6, 9, 0, 1, 2): 19,
 (6, 9, 0, 2, 3, 1): 4,
 (7, 6, 0, 1, 2, 1): 5,
 (7, 9, 0, 2, 2): 20,
 (7, 9, 1, 2, 3, 1): 6,
 (8, 9, 1, 2, 2): 21,
 (9, 6, 0, 2, 3, 1): 7,
 (9, 7, 1, 2, 3, 1): 8}

In [22]:
cohom.coker[(0,0)]

[ 1  0  0  0  0  0  0 -2  0  0  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0 -2  0  0  0  0  0  0  0  0]
[ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  1  0  0  2  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  1  0  0  0  0  0 -1  0  0  0  0  0  0]
[ 0  0  0  0  0  1 -1 -2  0  2  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]